# Single Atom Calculation

In [ ]:
# Uncomment the next line if you have ipympl installed and want interactive plots
# %matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

import pairinteraction.$PI_DTYPE as pi
from pairinteraction.visualization.colormaps import alphamagma

## Atom bases

In [ ]:
kets = [
    pi.KetAtom($SPECIES_0$QUANTUM_NUMBERS_0),
    pi.KetAtom($SPECIES_1$QUANTUM_NUMBERS_1),
]

bases = [
    pi.BasisAtom(kets[0].species$QUANTUM_NUMBERS_RESTRICTIONS_0),
    pi.BasisAtom(kets[1].species$QUANTUM_NUMBERS_RESTRICTIONS_1),
]

print(f"State of interest 1: {kets[0]}")
print(str(bases[0]))
print(f" ⇒ Basis 1 consists of {bases[0].number_of_kets} kets")

print(f"State of interest 2: {kets[1]}")
print(str(bases[1]))
print(f" ⇒ Basis 2 consists of {bases[1].number_of_kets} kets")

## Systems

In [ ]:
steps = $STEPS
efield_x = np.linspace($EX_MIN, $EX_MAX, steps)
efield_y = np.linspace($EY_MIN, $EY_MAX, steps)
efield_z = np.linspace($EZ_MIN, $EZ_MAX, steps)
bfield_x = np.linspace($BX_MIN, $BX_MAX, steps)
bfield_y = np.linspace($BY_MIN, $BY_MAX, steps)
bfield_z = np.linspace($BZ_MIN, $BZ_MAX, steps)
distance = np.linspace($DISTANCE_MIN, $DISTANCE_MAX, steps)
angle = np.linspace($ANGLE_MIN, $ANGLE_MAX, steps)

In [ ]:
systems_list = []
for step in range(steps):
    systems = tuple(
        pi.SystemAtom(bases[i])
        .set_electric_field([efield_x[step], efield_y[step], efield_z[step]], unit="V/cm")
        .set_magnetic_field([bfield_x[step], bfield_y[step], bfield_z[step]], unit="G")
        for i in range(2)
    )
    systems_list.append(systems)
systems_flattened = [system for systems in systems_list for system in systems]
pi.diagonalize(systems_flattened$DIAGONALIZE_KWARGS)

basis_pair_list = []
for step in range(steps):
    ket_pair_energy = sum(
        systems_list[step][i].get_corresponding_energy(kets[i], "GHz") for i in range(2)
    )
    basis_pair = pi.BasisPair(
        systems_list[step],
        energy=(ket_pair_energy - $PAIR_DELTA_ENERGY, ket_pair_energy + $PAIR_DELTA_ENERGY),
        energy_unit="GHz",
        m=$PAIR_M_RANGE,
    )
    basis_pair_list.append(basis_pair)

ket_pair_energy_0 = sum(systems_list[-1][i].get_corresponding_energy(kets[i], "GHz") for i in range(2))

In [ ]:
system_pair_list = []
for step in range(steps):
    system = pi.SystemPair(basis_pair_list[step])
    system.set_interaction_order($MULTIPOLE_ORDER)
    system.set_distance(distance[step], angle[step], unit="micrometer")
    system_pair_list.append(system)

pi.diagonalize(system_pair_list$DIAGONALIZE_KWARGS$DIAGONALIZE_ENERGY_RANGE_KWARGS)

In [ ]:
energies_list = [system.get_eigenenergies(unit="GHz") - ket_pair_energy_0 for system in system_pair_list]
overlaps_list = [system.get_eigenbasis().get_overlaps([kets[0], kets[1]]) for system in system_pair_list]

## Plot

In [ ]:
fig, ax = plt.subplots()

ax.set_xlabel($X_LABEL)
ax.set_ylabel("Energy [GHz]")

try:
    ax.plot($X_VARIABLE_NAME, np.array(energies_list), c="0.9", lw=0.25, zorder=-10)
except ValueError:  # inhomogeneous shape -> no simple line plot possible
    for x, es in zip($X_VARIABLE_NAME, energies_list):
        ax.plot([x] * len(es), es, c="0.9", ls="None", marker=".", zorder=-10)

x_repeated = np.hstack([val * np.ones_like(es) for val, es in zip($X_VARIABLE_NAME, energies_list)])
energies_flattend = np.hstack(energies_list)
overlaps_flattend = np.hstack(overlaps_list)
sorter = np.argsort(overlaps_flattend)

scat = ax.scatter(
    x_repeated[sorter],
    energies_flattend[sorter],
    c=overlaps_flattend[sorter],
    s=15,
    vmin=0,
    vmax=1,
    cmap=alphamagma
)

fig.colorbar(scat, ax=ax, label="Overlap with state of interest")

plt.show()